In [1]:
import os

In [2]:
%pwd

'/home/eman/Desktop/DataScience_proj/research'

In [ ]:
os.chdir("../") ## make our notebook access root directory

In [4]:
%pwd

'/home/eman/Desktop/DataScience_proj'

In [ ]:

from src.Datascience_proj import logger
import urllib.request  as request
import zipfile

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [ ]:
class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        ## read the file and assign to variable
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        ##creating our artifacts directory
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [35]:
## components -- Data ingestion component

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config

    ## Downloading the Zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url= self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Functons returns None
        """

        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-07-14 14:07:15,453: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-14 14:07:15,457: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-14 14:07:15,460: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-14 14:07:15,462: INFO: common: created directory at: artifacts]
[2025-07-14 14:07:15,463: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-14 14:07:15,464: INFO: 3313313224: File already exists]
